# 파이썬으로 배우는 기계학습
# Machine Learning with Python
**************

# 제 9-1 강: 역전파 2

----------

## 학습 목표
- 역전파 과정에서 오차함수의 미분을 학습한다.
- 오차 역전파로 각 층의 가중치를 조정한다.

## 학습 내용
- 은닉층과 출력층 사이 $∆𝑊^{[2]}$ 계산
- $𝑊^{[2]}$ 의 오차함수 미분
- $𝑊^{[1]}$ 의 오차함수 미분
- 역전파의 가중치 조정

-----------

### 인공 신경망 표기법

- 인공 신경망을 처음 공부할 때 반드시 통과해야 할 관문 중에 하나는 인공 신경망의 표기법입니다. 본 강의는 주로 Andrew Ng 교수의 표기법을 사용하고 있습니다. 
- Andrew Ng 교수의 표기법 한글 번역본이 [여기](http://taewan.kim/post/nn_notation/) 있으며, 강의
[Coursera: Neural Networks and Deep Learning by deeplearning.ai, 4Week: Deep Neural Networks - Deep L-layer neural network](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/7dP6E/deep-l-layer-neural-network)를 참고하길 바랍니다.

## 1. 오차 함수 미분  

지금 우리의 목표는 은닉층과 출력층 사이의 가중치를 조정하여 출력층에 나타난 오차를 줄이는 것입니다.  
지금까지 우리는 출력층에서의 오차 $E^{[2]}$를 활용하여 은닉층의 오차$E^{[1]}$를 구했는데, 이것은 나중에 $W^{[1]}$를 조정하기 위해 사용할 것입니다.  그러나, 우리는 이제 $\Delta W^{[2]}$를 계산하는데 집중하여 다음과 같은 $W^{[2]}$ 조정 공식을 완성하려는 것입니다.  참고로, $\Delta W^{[2]}$ 계산에는 $E^{[2]}$를 사용하게 될 것입니다.  

### 1단계: 가중치 $W^{[2]}$ 조정

\begin{align}
W^{[2]} &:= W^{[2]} - \alpha\Delta W^{[2]} \\ 
&= W^{[2]} - \alpha \frac{\partial E}{\partial W^{[2]}} \tag{1}
\end{align}

이런 공식은 어디서 생겨난 것일까요?  

전에 경사하강법을 공부한 것을 기억하나요? 같은 원리를 여기에 적용하고자 합니다. 경사하강법은 손실(오차)함수의 최소값을 찾아가는 방법입니다.  여기서 손실함수는 E는 가중치 W에 관한 함수입니다.  우리는 가중치 W를 조정함으로 E를 최소화하려는 것입니다.  이는 E는 W의 관한 함수 E(W)이기 때문에 가능합니다. 

예를 들면,  x에 관한 4차식 $f(x) = x^4 - 3x^3 + 2$가 있다면, 이 함수를 최소로 하는 x값은 어떻게 구할 수 있습니까?  $f(x)$를 그래프로 나타내면 다음의 왼쪽과 같습니다.  
<img src="images/gradientDescentPlot.png" width="6500">
<center>그림 1: 4차 함수의 최소값과 함수의 미분값</center>

$f(x)$를 미분한다는 의미는 $f(x)$ 함수의 $x$의 모든 지점에 대해 $f(x)$의 기울기를 구하는 것입니다. 기울기는 다른 말로 변화율입니다.  변화율은 $x$가 변할 때, $f(x)$가 얼마큼 변화하느냐를 비율로 나타내는 것입니다.  $f(x)$ 그래프를 살펴보면 대부분 모든 지점에서 $x$가 변함에 따라 $f(x)$도 변하는 것을 알 수 있습니다.  다만, $x$가 변해도 $f(x)$가 변하지 않거나 가장 적게 변하는 부분을 찾으면 됩니다. 그곳이 바로 $f(x)$의 미분값이 0가 되는 지점입니다. 

\begin{align}
f(x) &= x^4 - 3x^3 + 2 \\
f'(x) &= 4x^3 - 9x^2 \\
\end{align}

이 예제는 $f'(x) = 0$를 만족하는 $x$의 값이 두 곳이 있지만, 우리가 찾는 곳은 지역 최소값이 아니라 전역 최소값$^{global \ minimum}$의 경우입니다.  참고로 지역 최소값에서도 기울기가 0이 되는데, 이런 지점을 안장점$^{saddle \ point}$라고 합니다. 그러므로 함수 $f(x)$가 최소값을 갖는 정확한 $x$값은 $x = 2.25$ 입니다. 

식(1)을 살펴보면, $\alpha$는 학습률이며 대개 0과 1사이의 값이며, W도 정해진 값입니다.  여기서 $\Delta W$을 구하려고 하는데, 그 이유는 $W(E)$를 미분하여 기울기가 줄어드는 방향으로 조금씩 진행하려는 것입니다.  이것을 반복하면 결국에는 오차가 최소가 되는 지점까지 도달 할 수 있다는 것입니다. 

## 2. $W^{[2]}$의 오차함수 미분

식(1)에 표기 $W$ 행렬의 모든 원소들을 한번 다루는 것보다 원소 하나를 다루는 것이 훨씬 현실적입니다. 우선 은닉층 $j$와 출력층 $k$ 사이에 존재하는 연결 가중치만 고려한다면, 이 문제는 다음과 같이 축소됩니다. 

### 2단계: 가중치 $\Delta w_{jk}^{[2]}$ 조정
\begin{align}  
w_{jk}^{[2]} :&= w_{jk}^{[2]} - \alpha \Delta w_{jk}^{[2]} \\
             &= w_{jk}^{[2]} - \frac{\partial E}{\partial w_{jk}^{[2]}} \tag{2-1} 
\end{align}

식(2-1)을 영어로 표현하면 다음과 같습니다. 

This means, “the partial derivative of the error with respect to the weight from hidden node j to output node k.”

<img src="images/FinalFig1.PNG" width="500">
<center>그림 2: 역전파 오차함수 미분(1)</center>

식(2-2)는 가중치 $w_{jk}$의 값이 변함에 따라 오차 E의 값이 얼마만큼 변하는지 알려주는 식입니다.  

\begin{align}  \Delta w^{[2]}_{jk} = \frac{\partial E}{\partial w_{jk}} \tag{2-2} \end{align}

이 식의 값은 최저점의 방향으로 감소하기 원하는 오차함수의 기울기입니다.  오차함수 $E$는 가중치 $W$에 의해 발생하기 때문에, 오차함수 $E$는 $w_{jk}$에 관한 식이 될 것입니다.  

여기서 오차함수 $E$는 $n$개의 모든 출력 노드 $y_k (k = 1, ..., n)$에 대해 클래스 레이블 $y$와 출력값 $\hat{y}$의 차를 구하여 이를 제곱하여 모두 합한 것입니다.  전체 값을 반으로 나눈 것은 미분할 경우에 발생하는 2를 제거하기 위해 임의로 나눈 것이며, 이는 전체 계산에 영향을 미치지 않습니다. 

\begin{align} E = \sum_{n=1}^{n} \frac{1}{2} (y_n - \hat{y}_n)^2 \tag{3} \end{align}

식(3)을 식(2)에 대입할 수 있습니다. 

\begin{align} 
\frac{\partial E}{\partial w_{jk}} = \frac{\partial}{\partial w_{jk}} \frac{1}{2} \sum_{n=1}^{n} (y_n - \hat{y}_n)^2  \tag{4}
\end{align}

식(4)를 살펴보면, 출력 노드가 1에서 n까지 있을지라도 오직 $w_{jk}$ 와 연결된 노드와 연결가중치만이 오차에 서로 영향을 주고 받을 것입니다.  그러므로, $w_{jk}$와 연결되어 있지 않은 출력 노드는 식(4)에서 모두 제거할 수 있습니다.  오차함수를 구할 때 모든 출력 노드에 대해 오차를 합산할 필요가 없다는 것을 알았습니다.  그러면, 식(4)는 다음과 같이 간소화됩니다. 

\begin{align} 
\frac{\partial E}{\partial w_{jk}} = \frac{\partial}{\partial w_{jk}} \frac{1}{2} (y_k - \hat{y}_k)^2  \tag{5}
\end{align}

놀랍게도 모두 합하는 기호 $\sum$이 없어졌죠? 이제 미분에 도전할 만큼 간단해졌나요? 다음 미분 공식을 이용하면 간단합니다. 

\begin{align} \frac{\partial u^n}{\partial x} &= n u^{n-1} \frac{\partial u}{\partial x} \tag{5-1} \end{align}

시도해볼까요?

\begin{align} 
\frac{\partial E}{\partial w_{jk}} &= \frac{1}{2} \cdot 2 \ (y_k - \hat{y}_k) \frac{\partial}{\partial w_{jk}} (y_k - \hat{y}_k) \\
                                   &= (y_k - \hat{y}_k) \frac{\partial}{\partial w_{jk}} (- \hat{y}_k) \\
                                   &= -(y_k - \hat{y}_k) \frac{\partial \hat{y}_k}{\partial w_{jk}} \tag{6}
\end{align}

---------------
#### joywhy:

<span style="color:purple">
앞의 (6) 의 첫번째 줄에서 두번째 줄로 넘어갈 때, $y_k$ 항이 사라졌습니다. 왜 그럴까요?
</span>

\begin{align}  \frac{\partial y_k}{\partial w_{jk}} = 0 \end{align}
$y_k$는 무엇을 의미하나요? 

이것은 $k$번째 원소의 클래스 레이블입니다. 미분을 한다는 것은 변화율을 계산하는 것입니다. $w_{jk}$가 변할 때, $y_k$의 변화율을 계산하는 것입니다. 

그러면, $w_{jk}$가 변함에 따라 $y_k$값이 어떻게 변하나요? 

아니요. $y_k$값은 미리 정해진 목표값 즉 주어진 상수이므로, 변하지 않습니다. 그러므로 그의 미분 값은 0가 됩니다. 

------------

어느덧 미분해야 할 식이 단 하나의 항만 남아있게 되었습니다. 마지막 항  $\hat{y}_k$ 미분은 어떻게 하면 좋을까요? 

마지막 항 미분 역시 $\hat{y}_k$이 무엇을 의미하는지 알면 어려울 것이 없습니다. 
$\hat{y}_k$은 출력층 k노드의 출력 $y_k^{[2]}$입니다. 왜요? 우리가 그렇게 정의한 것이죠. 영어로 By definition 이라고 합니다.  
다른 말로, 출력층 노드 $k$의 결과 값이므로, 바로 앞 층(은닉층)에서 온 입력 신호의 가중치 합 $z_k^{[2]}$에 활성화 함수 $g(\cdot)$를 적용한 값이며 다음과 같이 계산한 것입니다. 

\begin{align}
  \hat{y}_k &= a_k^{[2]} \\
  a_k^{[2]} &= g(z_k^{[2]}) \\
  z^{[2]}_{k} &= \sum_j w^{[2]}_{jk}\cdot a^{[1]}_{j}  \tag{7}
\end{align}

여기서 $g(\cdot)$는 활성화 함수를 이며, $a^{[1]}_{j}$는 출력층의 출력이 아니라, 그 직전 노드 즉 은닉층 노드의 출력값인 것에 유의하십시오.  간략히 표기하기 위하여  $w_{jk}$와 $z_k$의 윗첨자 $[2]$를 생략하기로 하고, 식(6)을 다음과 같이 다시 쓸 수 있습니다.

### 3단계: 오차함수 미분  $\Delta w_{jk}^{[2]} = \frac{\partial E}{\partial w_{jk}}$ 

\begin{align}
\frac{\partial E}{\partial w_{jk}} = -(y_k - \hat{y}_k) \cdot  \frac{\partial }{\partial w_{jk}} g(z_k)  \tag{8-1}
\end{align}

식(8-1)의 마지막 항의 미분은 활성화 함수의 미분입니다. 그러면, 우리가 여기서 사용하고 있는 시그모이드 함수의 미분 값은 다음과 같습니다. 
\begin{align}
\frac{\partial \sigma(x)}{\partial x} = \sigma(x) \cdot (1 - \sigma(x))
\end{align}

식(8-1)에 시그모이드 미분을 적용하면, 다음과 같습니다. 마지막 항은 미분의 기본적인 공식 $\frac{\partial u^n}{\partial x} = n u^{n-1} \frac{\partial u}{\partial x}$ 에 의하여 나온 항입니다. 

\begin{align}
\frac{\partial E}{\partial w_{jk}} = -(y_k - \hat{y}_k)\cdot  
        \sigma(z_k) (1 -  \sigma(z_k) ) 
        \frac{\partial z_k} {\partial w_{jk}} \tag{8-2}
\end{align}

식(7)에서 기술한 바와 같이 $z^{[2]}_k = w^{[2]}_{jk}\cdot a^{[1]}_{j}$ 이므로, 이를 적용하면 다음과 같습니다.  

\begin{align}
\frac{\partial E}{\partial w_{jk}} &= -(y_k - \hat{y}_k) \cdot  
     \sigma(z_k)  (1 -  \sigma(z_k) ) \cdot
     \frac{\partial } {\partial w_{jk}} (\sum_j w_{jk}\cdot a_j) \\
     &= -(y_k - \hat{y}_k) \cdot \sigma(z_k) (1-\sigma(z_k) ) \cdot  a_j  \tag{9-1} 
\end{align}

여기서 $z_k$와 그의 미분은 다음과 같음을 유의하십시오.

\begin{align} z_k = z^{[2]}_k = \sum_j w^{[2]}_{jk}\cdot a^{[1]}_{j}  \end{align}

\begin{align} \frac{\partial } {\partial w_{jk}} \sum\limits_{j=1}^n w_{jk}\cdot a_j = \frac{\partial } {\partial w_{jk}}(w_{1k}a_1 + w_{2k}a_2 + ... + w_{jk}a_j + ... + w_{nk}a_n) = a_j \end{align}


식(9-1)으로부터 $\Delta W^{[2]}$ 대한 식도 다음과 같이 유추할 수 있습니다. 
\begin{align}
\Delta W^{[2]} &= \frac{\partial E}{\partial W^{[2]}} 
                = -(y - \hat{y}) \cdot A^{[2]} (1 - A^{[2]}) \cdot A^{[1]T} \tag{9-2} 
\end{align}

식(9)은 우리가 지금까지 찾으려고 달려왔던 수식입니다. 신경망을 학습시킬 알고리즘의 핵심입니다.  기계학습의 관문을 여는 열쇠입니다. 

이제 여러분 스스로 자축할 만 합니다. 

더 이상 어려운 수학은 없을 것입니다. 이제 이러한 수식을 응용하고 활용하여 코딩을 하면 됩니다.  코딩은 수식에 비해 훨씬 쉬울 것입니다. 

우리가 만들어낸 수식을 여유롭게 바라보며 다시 한번 그 맛을 천천히 음미해보길 바랍니다.

\begin{align} 
  \Delta w^{[2]}_{jk} &= \frac{\partial E}{\partial w_{jk}} 
          = -(y_k - \hat{y}_k) \cdot \sigma(z_k) (1-\sigma(z_k) ) \cdot  a_j  \\
  \Delta W^{[2]} &= \frac{\partial E}{\partial W^{[2]}} 
          = -E^{[2]} \cdot A^{[2]} (1 - A^{[2]}) \cdot A^{[1]T} \tag{9} 
\end{align}

- 첫 번째 항은 클래스 레이블과 신경망 출력값의 차이 즉 오차입니다. 
- 두 번째 항에 있는 $z_k$ 혹은 $Z^{[2]}$은 출력층에 들어오는 입력 신호입니다.  
- 두 번째 항 $\sigma(z_k) (1-\sigma(z_k))$ 혹은 $A^{[2]} (1 - A^{[2]})$은 활성화 함수를 미분$g'(\cdot)$하여 출력층의 입력 신호 $z_k$ 혹은 $Z^{[2]}$ 적용한 것입니다. 
- 마지막 항 $ a_j $은 이전 은닉층 노드 j의 결과 값 즉 출력입니다. 

궁극적으로 오차를 최소화하기 위하여 가중치를 조정할 때, 이러한 항목들이 영향을 미치고 있는 것입니다. 우리가 여기서 음미한 사항을 바탕으로 식(9)를 다음과 같이 간소화할 수 있습니다. 

### 4단계: 오차함수 미분  $\Delta w_{jk}^{[2]} = \frac{\partial E}{\partial w_{jk}}$ 

\begin{align} 
  \Delta w^{[2]}_{jk} &= -(y_k - \hat{y}_k) \cdot g' (z_k) \cdot  a_j  \\
  \Delta W^{[2]} &= -E^{[2]} \cdot g'(Z^{[2]}) \cdot A^{[1]T} \tag{9-3} 
\end{align}

--------
__joywhy__:
\begin{align} 
  \frac{\partial } {\partial w_{jk}} (\sum_j w_{jk}\cdot a_j) = a_j 
\end{align}

$\frac{\partial } {\partial w_{jk}} \sum_j w_{jk}$은 $w_{jk}$를 $w_{jk}$에 대해 미분하니까, 마치 x를 x에 대해 미분하는 것처럼 1이 됩니다. $a_j$는 출력층이 아니라 바로 앞층(은닉층)의 출력들입니다. 그러므로, $w_{ij}$가 변해도 $a_j$에는 아무런 영향을 끼치지 않으므로, $a_j$는 $w_{jk}$의 상수 계수입니다.  

-----------

### 5단계: 가중치$W^{[2]}$ 조정

\begin{align}
W^{[2]} :&= W^{[2]} - \alpha\Delta W^{[2]} \\ 
&= W^{[2]} - \alpha \frac{\partial E}{\partial W^{[2]}}  \\
&= W^{[2]} + \alpha E^{[2]} \cdot g'(Z^{[2]}) \cdot A^{[1]T}
\end{align}

## 3. $W^{[1]}$ 오차함수 미분
지금까지 우리가 구한 것은 은닉층과 출력층 사이의 가중치를 조정하기 위한 준비작업이었습니다. 이제 입력층과 은닉층 사이의 가중치를 조정하기 위한 오차 기울기를 구해야 합니다. 물론, 지금까지 작업해온 것과 아주 비슷할 것이라고 짐작할 수 있습니다. 






<img src="images/backpropDerivative2.png" width="500">
<center>그림 3: 역전파 오차함수 미분(2)</center>

식(9)에서 식(10)을 유추하면, 식(10)의  
- 첫번 째 항 $(e_j)$은 은닉층 노드에서 나오는 역전파된 오차를 합한 것입니다. 
- 두 번째 항$z_j$은 은닉층에 들어오는 입력 신호이고 $\sigma(z_j)$은 그 입력신호에 시그모이드 활성화 함수를 적용한 값입니다. 
- 마지막 항 $ a_i$은 첫 번째 층의 출력이므로 곧 입력 신호 $x_i$와 같습니다.   

\begin{align} \frac{\partial E}{\partial w_{jk}} = -(y_k - \hat{y_k}) \cdot 
\sigma(z_k) (1-\sigma(z_k) ) \cdot  a_j  \tag{9-1} \end{align}

\begin{align} \frac{\partial E}{\partial w_{ij}} = -(e_j) \cdot 
\sigma(z_j) (1-\sigma(z_j) ) \cdot  a_i  \tag{10-1} \end{align}

여기서 $z_j$는 다음과 같음을 유의하십시오.

\begin{align} z_j = z^{[1]}_j &= \sum_j w^{[1]}_{ij}\cdot a^{[0]}_j = \sum_j w^{[1]}_{ij}\cdot x_j \\
                              &= \mathbf{W^{[1]} \cdot A^{[0]}} = \mathbf{W^{[1]} \cdot X}                 \end{align}
                              
식(10-1)으로부터 $\Delta W^{[1]}$ 대한 식도 다음과 같이 유추할 수 있습니다. 
\begin{align}
\Delta W^{[1]} &= \frac{\partial E}{\partial W^{[1]}}  \\
               &= -E^{[1]} A^{[1]} (1 - A^{[1]}) A^{[0]T}  \tag{10-2}
\end{align}

식(10-1)과 식(10-2)를 좀 더 간략히 표현할 수 있습니다. 

\begin{align} \Delta w^{[1]}_{ij}  &= -(e_j) \cdot \sigma(z_j) (1-\sigma(z_j) ) \cdot  a_i  \\
                  &= -(e_j) \cdot g'(z_j) \cdot a_i   \\ \\
                 \Delta W^{[1]} &= -E^{[1]} A^{[1]} (1 - A^{[1]}) A^{[0]T}  \\ 
                 &= -E^{[1]} \cdot g'(Z^{[1]}) \cdot A^{[0]T}  \tag{10-3}
\end{align}

이제 기울기에 대한 모든 중요한 수식을 모두 구했기 때문에 이것을 이용하여 신경망의 모든 가중치를 조정할 수 있게 되었습니다. 

## 3. 역전파의 가중치 조정
 


우리가 오늘 강의를 시작할 때, 다음과 같이 가중치를 조정하는 식으로부터 출발하였습니다. 

\begin{align}
W^{[2]} &:= W^{[2]} - \alpha\Delta W^{[2]} \\ 
&= W^{[2]} - \alpha \frac{\partial E}{\partial W^{[2]}} \tag{1}
\end{align}

그리고, 이를 가중치 $w_{jk}$를 조정하기 위한 오차함수의 기울기를 구하였으므로, 이에 대한 조정을 다음과 같이 수학적으로 표현할 수 있습니다. 

\begin{align}
  new \ w_{jk} = old \ w_{jk} - \alpha \cdot \frac {\partial{E}}  {\partial w_{jk}} \tag{11}
\end{align}

새로운 가중치  $new \ w_{jk}$은 이전의 가중치 $old \ w_{jk}$에 우리가 계산한 오차 기울기에 부정을 취하여 조정한 가중치입니다.  부정$^{negative}$을 취한 이유는 양의 기울기가 있을 때에 가중치를 감소시키고 음의 기울기가 있을 때에 가중치를 증가시키기 위한 것입니다. 

우리가 이전에 경험한 것같이 가중치는 경사의 반대 방향으로 바뀌었다는 것을 기억합니다. 우리가 나쁜 학습 자료로 인해 최소값으로부터 너무 멀리 가는 것을 피하면서 동시에 가중치가 최소값을 계속 지나치지 않아야 합니다. 기호 알파 α는 최저치를 지나치지 않도록 하기 위해 변화의 정도를 조정하는 역할을 합니다. 이를 학습률$^{learning \ rate}$ 라고도 합니다.

시그모이드 활성화 함수를 사용할 경우, 식(1)을 행렬 형식으로 표현하면 다음과 같습니다. 

\begin{align}
W^{[2]} \; &+ \!= \; \alpha * E^{[2]} * A^{[2]} \cdot (1 - A^{[2]}) \cdot A^{[1]T}  \\
W^{[1]} \; &+ \!= \; \alpha * E^{[1]} * A^{[1]} \cdot (1 - A^{[1]}) \cdot A^{[0]T}  \tag{12-1}
\end{align}

일반화된 식으로 나타나면, 식(1)을 행렬 형식으로 표현하면 다음과 같습니다. 

\begin{align}
W^{[2]} \; &+ \!= \; \alpha * E^{[2]} * g'(Z^{[2]}) \cdot A^{[1]T}  \\
W^{[1]} \; &+ \!= \; \alpha * E^{[1]} * g'(Z^{[1]}) \cdot A^{[0]T}  \tag{12-2}
\end{align}

여기서 오차 $E^{[1]}$는 앞 강의에서 다룬 것처럼 다음과 같이 계산합니다. 
\begin{align}
E^{[2]} &= y - \hat{y} \\
E^{[1]} &= W^{[2]T} \cdot E^{[2]}
\end{align}

----------
#### joywhy: 

식(9-2), 식(10-2) 혹은 식(12)에서 A 행렬의 전치는 어디서 나왔나요?

식(9-1)의 마지막 유도 과정을 다시 살펴봅시다. 
\begin{align}
\frac{\partial E}{\partial w_{jk}} &= -(y_k - \hat{y}_k) \cdot  
     \sigma(z_k)  (1 -  \sigma(z_k) ) \cdot
     \frac{\partial } {\partial w_{jk}} (\sum_j w_{jk}\cdot a_j) \\
     &= -(y_k - \hat{y}_k) \cdot \sigma(z_k) (1-\sigma(z_k) ) \cdot  a_j  \tag{9-1} 
\end{align}

마지막 항을 미분하여 나타난 $a_j$는 $[a^{[1]}_1 \ a^{[1]}_2 \ ... \ a^{[1]}_n]$이므로, 이는 곧 $A^{[1]T}$ 입니다.  $j$는 은닉층의 노드 수 $n$입니다.  다른 행렬들의 전치도 이와 비슷한 과정에서 나온 것입니다. 

---------

### joycore:

- 신경망의 오차는 가중치의 함수입니다.
- 신경망을 개선한다는 것은 가중치를 조정하여 오차를 줄이는 것입니다.
- 최적의 가중치를 대수적으로 계산하는 것은 어렵습니다. 다른 방법은 오차 함수에서 오차를 감소하는 방향으로 가중치를 반복적으로 조금씩 개선하는 것입니다.  이를 경사 하강법이라 합니다.
- 이 오차 기울기는 미분을 사용하여 계산할 수 있습니다.


### 참고자료: $W^{[2]}$의 오차함수 미분

\begin{align}
\Delta W^{[2]} & = \frac{\partial E}{\partial W^{[2]}} \qquad by \ chain \ rule \\ 
& = \frac{\partial E}{\partial A^{[2]}}\frac{\partial A^{[2]}}{\partial W^{[2]}} \\ 
& =  \frac{\partial E}{\partial A^{[2]}}\frac{\partial A^{[2]}}{\partial Z^{[2]}}\frac{\partial Z^{[2]}}{\partial W^{[2]}}    \qquad \because \ E = \frac{1}{2}(\hat{y} - y)^2 \; and \; A^{[2]} = \hat{y} \\ 
& =  (\hat{y}-y)\frac{\partial A^{[2]}}{\partial Z^{[2]}}\frac{\partial Z^{[2]}}{\partial W^{[2]}}   \qquad \because \ A^{[2]} = g(Z^{[2]})    \\ 
& =  (\hat{y}-y)g’(Z^{[2]})\frac{\partial Z^{[2]}}{\partial W^{[2]}} \qquad \because \ Z^{[2]} = W^{[2]} \cdot A^{[1]}    \\ 
& =  (\hat{y}-y)g’(Z^{[2]}) A^{[1]T}   \\ 
& =  (\hat{y}-y)sigmoid'(Z^{[2]}) A^{[1]T} \qquad if \ g(\cdot) = sigmoid(\cdot)  \\ 
& =  (\hat{y}-y)A^{[2]}(1-A^{[2]})A^{[1]T}
\end{align}

---------------
__joywhy__:  
\begin{align}   \frac{\partial E}{\partial A^{[2]}} =  \hat{y}-y \tag{a} \end{align}

신경망의 최종 결과는 $\hat{y}$으로 표기하며, 이는 $A^{[2]}$와 같습니다. 

\begin{align}   \hat{y} = A^{[2]} \tag{b} \end{align} 

신경망의 클래스 레이블은 y로 표기하며, 이는 주어진 상수입니다. 

신경망의 오차 $E$는 다음과 같이 계산할 수 있으며, 여기서 $\frac{1}{2}$는 미분 값을 간소화하기 위하여 추가한 항목이며, 우리의 오차 기울기를 계산하는데는 영향을 미치지 않습니다. 

\begin{align} E = \frac{1}{2}(\hat{y} - y)^2  \tag{c} \end{align} 

식(b)와 식(c)를 (a)에 대입하면 식(a)는 다음과 같이 미분할 수 있습니다. 

\begin{align}   
  \frac{\partial E}{\partial A^{[2]}} &=  \frac{\partial} {\partial \hat{y}} \big[ \frac{1}{2}(\hat{y} - y)^2 \big ] \\
   &= \frac{\partial} {\partial \hat{y}} \big[ \frac{1}{2}(\hat{y}^2 - 2\hat{y}y + y^2) \big] = \hat{y}-y \\
   or \\
   &= \frac{1}{2} * 2 * (\hat{y} - y) \frac{\partial (\hat{y} - y)} {\partial \hat{y} } = \hat{y}-y \\
\end{align}


-------------------

__JoyQuiz 1__
다음과 같은 식에서 옳지 않은 것을 두 개 선택하십시오.

(1) $\hat{y} = A^{[2]}  $

(2) $Z^{[2]} = W^{[2]} \cdot A^{[2]} $

(3) $ \frac{\partial E}{\partial A^{[2]}} =  \hat{y}-y $

(4) $  A^{[2]} = g(Z^{[2]}) $

(5) $ \frac{\partial } {\partial w_{jk}} (\sum_j w_{jk}\cdot a_j) = a_j $

(6) $ \frac{\partial }{\partial w_{jk}} g(z_k) = \sigma(z_k) (1-\sigma(z_k) ) $

__JoyAnswer__ 

(2) $Z^{[2]} = W^{[2]} \cdot A^{[1]}$

(6) $ \frac{\partial }{\partial w_{jk}} g(z_k) = \sigma(z_k) (1-\sigma(z_k) ) \quad if \ g(x) = \sigma(x) $